In [2]:
import numpy as np
import ctypes as ct
import glob, os

import os.path
from os import path

import tkinter
from tkinter import messagebox

import run_routing

import CreationParam
import ExecProg
import TraceChemins
import TraceFondCarte

# Bokeh libraries
from bokeh.io import output_file
from bokeh.plotting import figure, show
from bokeh.models import Range1d
from bokeh.models.widgets import Panel, Tabs, Toggle, TextInput
from bokeh.layouts import column, row  , layout
from bokeh.models import CustomJS, Div, Button
from bokeh.events import ButtonClick
from bokeh.io import curdoc
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Select
from bokeh.models import HoverTool
from bokeh.layouts import gridplot
from bokeh.models.widgets import Paragraph


source = "/home/nathan/TAL/"
chemin = "/home/nathan/TAL/Routage/Sources/NOISY_PAES/"
cheminMeteo = "/home/nathan/TAL/grib/"
cheminPolar = "/home/nathan/TAL/polar/"

nbExec = 0
#******************************************************************************************************
#Création graph 

# On choisis les outils de la barre d'outils
#select_tools = ['box_zoom']

# Specify the tools
select_tools = ['lasso_select', 'tap', 'reset', 'save', 'box_zoom']


# On créer la figure
figChemins = figure(title='Graph meilleurs chemins',
             plot_height=750, plot_width=750,
             x_axis_label='Longitude', y_axis_label='Latitude', 
             toolbar_location="right",tools=select_tools, 
             match_aspect=True)

#********************************************************************************************************
#Création du 2nd graph

# On créer la figure
figPoints = figure(title='Graph isofront',
             plot_height=400, plot_width=400,
             x_axis_label='Time', y_axis_label='Stress', 
             toolbar_location="right",tools=select_tools)

#*******************************************************************************************************
#Menu

#differente technique pour récuperer le nom des fichiers sans avoir besoin de passer par des fichier .txt
#ne fonctionne pas


#on créer les widgets
laD = TextInput(value="47.29891", title="Lat depart :")
loD= TextInput(value="-3.57457", title="Long depart :")
laA = TextInput(value="44.28568", title="Lat arrivee :")
loA = TextInput(value="-5.32721", title="Long arrivee :")
date = TextInput(value="20151005", title="Date :")
stime = TextInput(value="185544", title="Stime :")


'''
#TEST CARTE 
#on créer les widgets
laD = TextInput(value="49.591995", title="Lat depart :")
loD= TextInput(value="-6.614903", title="Long depart :")
laA = TextInput(value="46.422050", title="Lat arrivee :")
loA = TextInput(value="0.453375", title="Long arrivee :")
date = TextInput(value="20151005", title="Date :")
stime = TextInput(value="185544", title="Stime :")
'''


#on recupere les noms des fichiers polar disponible : 
tabPolar = []
for file in glob.glob(cheminPolar+"*.xml"):  
    file = file.replace(cheminPolar+'','')
    file = file.replace('.xml','')
    
    tabPolar.append(file)
    
polar = Select(title="Polar:", value=tabPolar[0], options=tabPolar)   


#on récupere les noms des fichiers meteo disponible : 
tabWeather = []
for file in glob.glob(cheminMeteo+"gribfile/*.grb"): 
    file = file.replace(cheminMeteo+'gribfile/','')
    file = file.replace('.grb','')
    
    tabWeather.append(file)
    
weather = Select(title="Weather:", value=tabWeather[0], options=tabWeather)   



button = Button(label="Lancer programme", button_type="success")
button1 = Button(label="Modifier params secondaires")
par = Paragraph(text="""""",style={'color': 'red'})


#les widgets "secondaires" sont désactivé au lancement de l'application
seed = TextInput(value="5", title="Seed :",disabled=True)
noise = TextInput(value="0.0", title="Noise :",disabled=True)
gens = TextInput(value="4000", title="Gens :",disabled=True)
alpha = TextInput(value="100", title="Alpha :",disabled=True)
depth = TextInput(value="4", title="depth :",disabled=True)


#*******************************************************************************************************
#CALLBACKS

def creationParam():
    #on récupere les differentes valeurs 
    Seed = "seed 5"
    start = "start "+laD.value+" "+loD.value
    finish = "finish "+laA.value+" "+loA.value
    Date = "date "+date.value
    Stime = "stime "+stime.value
    
    Polar = "polar ../"+polar.value+".dat"
    Weather = "weather ../"+weather.value+".isoc" 
    
    Noise = "noise 0.0"
    Gens = "gens 4000"
    Alpha = "alpha 100"
    Depth = "depth 4"
    outputfile = "outputfile output.out"
    
    CreationParam.creationParam(Seed,start,finish,Date,Stime,Polar,Weather,Noise,Gens,Alpha,Depth,outputfile)
    
    
def lancerProg():
    ExecProg.polar(cheminPolar, polar.value)
    ExecProg.grib(cheminMeteo, weather.value) 
    
    longmax = max(float(laD.value),float(laA.value)) + 10
    longmin = min(float(laD.value),float(laA.value)) - 10
    latmax = max(float(loD.value),float(loA.value)) + 10
    latmin = min(float(loD.value),float(loA.value)) - 10 

    ExecProg.carto(source, longmax, longmin, latmax, latmin)  
    ExecProg.calculRoutes(chemin)

    
def MiseAJourGraph(val):   
    
    #*******************************************************************************************
    #Fond de carte :    
    os.chdir(source)
    
    arrayGlobalX = []
    arrayGlobalY = []
    
    TraceFondCarte.Carto(source,arrayGlobalX,arrayGlobalY) 
    figChemins.multi_line(xs=arrayGlobalX, ys=arrayGlobalY)
    
    #******************************************************************************************
    #tracés des chemins :
    
    os.chdir(chemin)
      
    
    #*******************************************************************************************
    #le meilleur chemin : 
    arrayLat = []
    arrayLong = []
    arrayDist = []
    arrayTime = []
    arrayStress = []
    arrayTimeGlobale = []
    arrayStressGlobale = [] 
    
    TraceChemins.MeilleurChemin(arrayLat,arrayLong,arrayDist,arrayTime,arrayStress,
                                arrayTimeGlobale,arrayStressGlobale)
    
    
    sourceMeilleurChemin = ColumnDataSource(data=dict(x=arrayLong, y=arrayLat, dist=arrayDist, 
                                         time=arrayTime, stress=arrayStress, 
                                         x2=arrayTimeGlobale, y2=arrayStressGlobale)) 
    
    #on creer la ligne du meilleur chemin
    figChemins.line('x','y',source=sourceMeilleurChemin, 
                color='blue', line_width=3,
                legend='Meilleur chemin')

    # Draw the coordinates as circles
    figPoints.circle('x2', 'y2',source=sourceMeilleurChemin,
                color='blue', size=10, alpha=0.5,
                legend='Meilleur chemin')

    
    
    #**********************************************************************************************
    #autres chemin :
    
    tabChemin = []

    TraceChemins.AutresChemins(tabChemin)
    
    #on trace les chemins en parcourant le tableau             
    for i in range(len(tabChemin)):
        nom = 'line'+str(i)
        figChemins.line('x', 'y', source=tabChemin[i], color='black', line_width=0.5, name=nom)            
                        
                    
    #on place les points en parcourant le tableau             
    for i in range(len(tabChemin)):
        nom = 'point'+str(i)
        figPoints.circle('x2', 'y2',source=tabChemin[i],
                    color='black', size=5, alpha=0.5, name=nom)                
              
    
#****************************************************************************************************
    
def program():
    
    global nbExec
    
    nbExecLocal = nbExec 
    nbExec = nbExec + 1
    
    creationParam()
    lancerProg()
    MiseAJourGraph(nbExecLocal)
    
    # Format the tooltip
    tooltips = [
                ('dist to dest', '@dist'),
                ('time to dest', '@time'),
                ('stress global', '@stress'),
               ]
    
    tooltips2 = [
            ('stress', '@y2'),
            ('time', '@x2'),
           ]
    
    # Add the HoverTool to the figure
    figChemins.add_tools(HoverTool(tooltips=tooltips))
    figPoints.add_tools(HoverTool(tooltips=tooltips2))  
    
def hide():
    if seed.disabled==True :
        seed.disabled=False
        noise.disabled=False
        gens.disabled=False
        alpha.disabled=False
        depth.disabled=False
        button1.label="Bloquer params secondaires"
    else :
        seed.disabled=True
        noise.disabled=True
        gens.disabled=True
        alpha.disabled=True
        depth.disabled=True
        button1.label="Modifier params secondaires"

#******************************************************************************************************    

button.on_click(program) 
button1.on_click(hide)

#*********************************************************************************************************
#Organisation de la page

controls1 = [laD,laA,date,polar,button,par,button1,seed,noise,gens,alpha,depth]
controls2 = [loD,loA,stime, weather,figPoints]

inputs1 = column(*controls1, width=400, height=500)
inputs2 = column(*controls2, width=400, height=500)

controls3 = [inputs1,inputs2, figChemins]

inputs3 = row(*controls3, width=100, height=100)

l = layout([
    [inputs3],
], sizing_mode="scale_both")


curdoc().add_root(l)
curdoc().title = "Application routage voilier"